In [1]:
def text():
    f1=open('good.txt','r',encoding='utf-8')
    f2=open('bad.txt','r',encoding='utf-8')
    line1=f1.readline()
    line2=f2.readline()
    str=''
    while line1:
        str+=line1
        line1=f1.readline()
    while line2:
        str+=line2
        line2=f2.readline()
    f1.close()
    f2.close()
    return str
#单个字作为特征
def bag_of_words(words):
    return dict([(word,True) for word in words])
#print(bag_of_words(text())

#把词语（双字）作为搭配，并通过卡方统计，选取排名前1000的词语
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
def bigram(words, score_fn=BigramAssocMeasures.chi_sq, n=1000):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)  # 使用卡方统计的方法，选择排名前1000的词语
    newBigrams = [u + v for (u, v) in bigrams]
    #return bag_of_words(newBigrams)
#print(bigram(text(),score_fn=BigramAssocMeasures.chi_sq,n=1000))

# 把单个字和词语一起作为特征
def bigram_words(words, score_fn=BigramAssocMeasures.chi_sq, n=1000):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    newBigrams = [u + v for (u, v) in bigrams]
    a = bag_of_words(words)
    b = bag_of_words(newBigrams)
    a.update(b)  # 把字典b合并到字典a中
    return a  # 所有单个字和双个字一起作为特征
#print(bigram_words(text(),score_fn=BigramAssocMeasures.chi_sq,n=1000))

import jieba
# 返回分词列表如：[['我','爱','北京','天安门'],['你','好'],['hello']]，一条评论一个
def readfile(filename):
    stop=[line.strip() for line in open('stop.txt','r',
                                        encoding='utf-8').readlines()]
    f=open(filename,'r',encoding='utf-8')
    line=f.readline()
    str=[]
    while line:
        s=line.split('\t')
        fenci=jieba.cut(s[0],cut_all=False)
        str.append(list(set(fenci)-set(stop)-set(['\ufeff','\n'])))
        line=f.readline()
    return str

from nltk.probability import FreqDist,ConditionalFreqDist
from nltk.metrics import BigramAssocMeasures
# 获取信息量较高(前number个)的特征(卡方统计)
def jieba_feature(number):
    posWords=[]
    negWords=[]
    for items in readfile('good.txt'):
        for item in items:
            posWords.append(item)
    for items in readfile('bad.txt'):
        for item in items:
            negWords.append(item)
    word_fd=FreqDist()   # 可统计所有词的词频
    con_word_fd=ConditionalFreqDist()    # 可统计积极文本中的词频和消极文本中的词频
    for word in posWords:
        word_fd[word]+=1
        con_word_fd['pos'][word]+=1
    for word in negWords:
        word_fd[word]+=1
        con_word_fd['neg'][word]+=1
    pos_word_count=con_word_fd['pos'].N()    # 积极词的数量
    neg_word_count=con_word_fd['neg'].N()    # 消极词的数量
    # 一个词的信息量等于积极卡方统计量加上消极卡方统计量
    total_word_count=pos_word_count+neg_word_count
    word_scores={}
    for word,freq in word_fd.items():
        pos_score=BigramAssocMeasures.chi_sq(con_word_fd['pos'][word],(freq,
                                                                       pos_word_count),total_word_count)
        neg_score=BigramAssocMeasures.chi_sq(con_word_fd['neg'][word],(freq,
                                                                       neg_word_count),total_word_count)
        word_scores[word]=pos_score+neg_score
        best_vals=sorted(word_scores.items(),key=lambda item: item[1],
                         reverse=True)[:number]
        best_words=set([w for w,s in best_vals])
    return dict([(word,True) for word in best_words])
# 构建训练需要的数据格式：
# [[{'买': 'True', '京东': 'True', '物流': 'True', '包装': 'True', '\n': 'True', '很快': 'True', '不错': 'True', '酒': 'True', '正品': 'True', '感觉': 'True'},  'pos'],
# [{'买': 'True', '\n':  'True', '葡萄酒': 'True', '活动': 'True', '澳洲': 'True'}, 'pos'],
# [{'\n': 'True', '价格': 'True'}, 'pos']]
def build_features():
    #feature = bag_of_words(text())
    #feature = bigram(text(),score_fn=BigramAssocMeasures.chi_sq,n=900)
    # feature =  bigram_words(text(),score_fn=BigramAssocMeasures.chi_sq,n=900)
    feature = jieba_feature(1000)  # 结巴分词
    posFeatures = []
    for items in readfile('good.txt'):
        a = {}
        for item in items:
            if item in feature.keys():
                a[item] = 'True'
        posWords = [a, 'pos']  # 为积极文本赋予"pos"
        posFeatures.append(posWords)
    negFeatures = []
    for items in readfile('bad.txt'):
        a = {}
        for item in items:
            if item in feature.keys():
                a[item] = 'True'
        negWords = [a, 'neg']  # 为消极文本赋予"neg"
        negFeatures.append(negWords)
    return posFeatures, negFeatures


In [2]:
posFeatures,negFeatures=build_features()
from random import shuffle
shuffle(posFeatures)    #把文本的排列随机化
shuffle(negFeatures)
train=posFeatures[200:]+negFeatures[200:]   #只在1000条数据时合适，二八原则
test=posFeatures[:200]+negFeatures[:200]
data,tag=zip(*test)    #分离测试集合的数据和标签，便于测试


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\11514\AppData\Local\Temp\jieba.cache
Loading model cost 4.748 seconds.
Prefix dict has been built successfully.


In [5]:
from nltk.classify.scikitlearn import SklearnClassifier
def score(classifier):
    classifier=SklearnClassifier(classifier)
    classifier.train(train)
    pred=classifier.classify_many(data)
    n=0
    s=len(pred)
    for i in range(0,s):
        if(pred[i]==tag[i]):
            n=n+1
    return n/s
#通过实验，比较预测准确度score进而得出最佳特征提取方式、最佳特征维度、最佳分类算法
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
#print('LogisticRegression`s accuracy is  %f' % score(LogisticRegression()))
#处理输入的评论文本，使其成为可预测格式
def build_page(page):
    #四中特征提取方式
    # feature1 = bag_of_words(text())
    # n为特征维度，可调整
    # feature2 = bigram(text(),score_fn=BigramAssocMeasures.chi_sq,n=1000)
    # feature 3=  bigram_words(text(),score_fn=BigramAssocMeasures.chi_sq,n=1000)
    feature4 = jieba_feature(1000)  # 结巴分词，选取1000为特征维度，可调整
    temp={}
    '''
     #单个字为特征
     for word in page:
         if word in feature1:
             temp[word]='True'
     #双字为特征
     bigrams= BigramCollocationFinder.from_words(words)
     text=[u + v for (u, v) in bigrams.ngram_fd]
     for words in text:
         if words in feature2:
             temp[words]='True'
     #单字和双字为特征
     bigrams= BigramCollocationFinder.from_words(words)
     text=[u + v for (u, v) in bigrams.ngram_fd]
     for word in page:
         text.append(word)
     for words in text:
         if words in feature3:
             temp[words]='True'
      '''
    #现采用结巴分词形式处理待测文本
    fenci0=jieba.cut(page,cut_all=False)
    stop=[line.strip() for line in open('stop.txt','r',
                                        encoding='utf-8').readlines()]   #停用词
    for words in list(set(fenci0)-set(stop)):
        if words in feature4:
            temp[words]='True'
    return temp


In [6]:
#将实验比较得出的最佳分类算法（classifier_ag）构造的分类器保存
def classfier_model(classifier_ag):
    classifier = SklearnClassifier(classifier_ag)
    classifier.train(train)
    return classifier
#假设逻辑回归为最佳分类算法
#classifier=classfier_model(classifier_ag=LogisticRegression)
#用最佳分类器预测待测文本
def predict_page(page):
    pred = classifier.classify_many(data)
    return pred


In [7]:
#对users中的每条评论进行情感判断
def emotion_decide():
    results=collection1.find()
    for result in results:
        if result['page']:   #若评论为空，则不进行情感分析
           condition={'page':result['page']}
           if collection1.update_many(condition,{'$set':{
               'emotion':emotional.predict_page(page=result['page'])}}):
               print("successful")

